In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta 
from sklearn.preprocessing import StandardScaler
import json
from scipy import stats
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score

In [2]:
dataset = pd.read_csv("Motion data.csv", header=0)

In [3]:
#drop data that are anomalous or lying down (id 13, 15, 17, 26, 30, 39)

def checkIfInList(lst, value):
    for item in lst:
        if item == value:
            return True 
    return False 

anomaly_index = [13, 26, 30, 39]
anomaly_lst_index = dataset[(dataset['id'] == 13) | (dataset['id'] == 15) | (dataset['id'] == 17) | (dataset['id'] == 26) | (dataset['id'] == 30) | (dataset['id'] == 39)].index
dataset_cleaned = dataset.drop(anomaly_lst_index)
dataset_cleaned

,id,date,gyroX,gyroY,gyroZ,accelX,accelY,accelZ,activity,uuid
0,1,Mon Oct 25 2021,"[0, 499.664306640625, 499.8092651367187, 0.099...","[0, 1.25885009765625, 1.556396484375, 1.716613...","[0, 1.64031982421875, 1.38092041015625, 1.4953...","[0, 1.220703125, 1.1845703125, 1.185546875, 1....","[0, 1.1669921875, 1.14453125, 1.1640625, 1.186...","[0, 3.64453125, 3.673828125, 3.66796875, 3.655...",Working,NaN
1,2,Mon Oct 25 2021,"[0, 499.45068359375, 0.30517578125, 0.12969970...","[0, 2.02178955078125, 2.3193359375, 1.54876708...","[0, 1.4495849609375, 1.91497802734375, 1.76239...","[0, 0.4560546875, 0.4501953125, 0.4443359375, ...","[0, 13.169921875, 13.1591796875, 13.1767578125...","[0, 2.6796875, 2.6953125, 2.630859375, 2.66308...",Working,NaN
2,3,Mon Oct 25 2021,"[0, 499.969482421875, 0.2288818359375, 499.931...","[0, 1.01470947265625, 0.6866455078125, 1.73950...","[0, 1.45721435546875, 1.6632080078125, 1.48773...","[0, 0.5498046875, 0.6025390625, 0.568359375, 0...","[0, 15.5771484375, 15.5791015625, 15.58984375,...","[0, 3.904296875, 3.85546875, 3.93359375, 3.899...",Working,NaN
3,4,Mon Oct 25 2021,"[0, 496.673583984375, 486.8316650390625, 478.0...","[0, 25.115966796875, 39.9169921875, 498.283386...","[0, 493.01910400390625, 497.29156494140625, 49...","[0, 1.6650390625, 1.0244140625, 1.0927734375, ...","[0, 1.4375, 1.4013671875, 1.046875, 0.60253906...","[0, 3.4765625, 3.51953125, 3.548828125, 4.0439...",Working,NaN
4,6,Mon Oct 25 2021,"[0, 499.22943115234375, 499.5574951171875, 0.0...","[0, 4.63104248046875, 7.05718994140625, 4.6234...","[0, 0.6561279296875, 1.88446044921875, 2.47192...","[0, 1.3173828125, 1.3486328125, 1.3134765625, ...","[0, 1.1796875, 1.1953125, 1.166015625, 1.14746...","[0, 3.6904296875, 3.6669921875, 3.662109375, 3...",Working,NaN
5,7,Mon Oct 25 2021,"[0, 499.8779296875, 498.7640380859375, 498.435...","[0, 1.53350830078125, 499.3133544921875, 0.610...","[0, 1.007080078125, 3.11279296875, 1.342773437...","[0, 0.638671875, 0.67578125, 0.611328125, 0.57...","[0, 2.4521484375, 2.39453125, 2.435546875, 2.3...","[0, 3.1640625, 3.1787109375, 3.154296875, 3.20...",Walking,NaN
6,9,Mon Oct 25 2021,"[0, 7.110595703125, 6.40869140625, 0.373840332...","[0, 471.1761474609375, 489.68505859375, 472.83...","[0, 6.49261474609375, 30.1666259765625, 54.016...","[0, 2.1142578125, 2.310546875, 1.6328125, 0.08...","[0, 12.9677734375, 12.875, 12.6318359375, 12.8...","[0, 13.37890625, 13.0986328125, 12.853515625, ...",Running,NaN
7,11,Mon Oct 25 2021,"[0, 498.5809326171875, 498.2757568359375, 498....","[0, 2.5177001953125, 1.1749267578125, 1.266479...","[0, 0.457763671875, 0.20599365234375, 0.267028...","[0, 15.587890625, 15.5869140625, 15.607421875,...","[0, 0.359375, 0.369140625, 0.369140625, 0.3613...","[0, 4.3134765625, 4.2939453125, 4.2998046875, ...",Working,NaN
10,16,Mon Oct 25 2021,"[0, 418.3425903320313, 341.43829345703125, 321...","[0, 27.28271484375, 34.149169921875, 470.06988...","[0, 20.5535888671875, 36.91864013671875, 43.39...","[0, 15.873046875, 0.0859375, 0.0185546875, 0.0...","[0, 3.97265625, 3.3330078125, 2.4326171875, 2....","[0, 1.8212890625, 1.6162109375, 2.2265625, 3.3...",Working,NaN
12,18,Tue Oct 26 2021,"[0, 472.26715087890625, 454.6051025390625, 487...","[0, 39.75677490234375, 22.5372314453125, 485.4...","[0, 382.3776245117187, 439.88037109375, 39.550...","[0, 4.431640625, 4.3427734375, 5.0859375, 3.75...","[0, 1.4013671875, 1.373046875, 1.48046875, 0.8...","[0, 15.09765625, 15.1484375, 14.96484375, 15.4...",Walking,F0:F8:F2:86:96:85


In [4]:
#function to break data to 30 timesteps 
X = []
y = []

def breakIntoChunks(lst, size, label):
    lst = lst.tolist()
    global X
    global y
    numChunks = int(len(lst) / size)
    for i in range(0, numChunks):
        firstIndex = 30 * i
        secondIndex = 30 * (i+1)
        chunk = lst[firstIndex: secondIndex]
        chunk = np.array(chunk)
        chunk = np.reshape(chunk, (-1, 6))
        X.append(chunk)
        y.append(label)
    return

In [5]:
#convert data into 30 timesteps (3s)
for index, row in dataset_cleaned.iterrows():
    
    #convert to list 
    gyroX_set = json.loads(row['gyroX'])
    gyroY_set = json.loads(row['gyroY'])
    gyroZ_set = json.loads(row['gyroZ'])
    accelX_set = json.loads(row['accelX'])
    accelY_set = json.loads(row['accelY'])
    accelZ_set = json.loads(row['accelZ'])
    
    #stack into an array
    row_data = np.vstack((gyroX_set, gyroY_set, gyroZ_set, accelX_set, accelY_set, accelZ_set)).transpose()
    
    breakIntoChunks(row_data, 30, row['activity'])

In [6]:
#to numpy
X_arr = np.array(X)
y_arr = np.array(y)

In [7]:
#split with train test split 
trainvalX, testX, trainvalY, testY = train_test_split(X_arr, y_arr, test_size=0.1, random_state = 42)

In [8]:
trainX, valX, trainY, valY = train_test_split(trainvalX, trainvalY, test_size=0.15, random_state=42)

In [9]:
#one hot encoder 

#reshape array
reshape_trainY = trainY.reshape(len(trainY), 1)
reshape_valY = valY.reshape(len(valY), 1)
reshape_testY = testY.reshape(len(testY), 1)

encoder = OneHotEncoder(sparse=False).fit(reshape_trainY)
onehot_trainY = encoder.transform(reshape_trainY)
onehot_valY = encoder.transform(reshape_valY)
onehot_testY = encoder.transform(reshape_testY)

In [10]:
def mapResultsToOneHot(results):
    output = []
    for vector in results:
        vector_lst = vector.tolist()
        max_Index = vector_lst.index(max(vector_lst))
        onehot_vector = []
        for i in range(0, len(vector_lst)):
            if i == max_Index:
                onehot_vector.append(1)
            else:
                onehot_vector.append(0)
        output.append(onehot_vector)
    return output

In [11]:
from keras.models import Model, load_model
#load model 
model2 = load_model('Physical Activity Model/savedModel_stackedLSTM')

In [12]:
#evaluate performance 

#training set 
trainPredict2 = model2.predict(trainX)
trainPredict2 = mapResultsToOneHot(trainPredict2)
trainPredict2 = encoder.inverse_transform(trainPredict2)
trainPredict2 = trainPredict2.reshape(-1,)
trainAccuracy2 = accuracy_score(trainY, trainPredict2)
print("Accuracy of training set %f" % trainAccuracy2)

#validation set 
valPredict2 = model2.predict(valX)
valPredict2 = mapResultsToOneHot(valPredict2)
valPredict2 = encoder.inverse_transform(valPredict2)
valPredict2 = valPredict2.reshape(-1,)
valAccuracy2 = accuracy_score(valY, valPredict2)
print("Accuracy of validation set %f" % valAccuracy2)

Accuracy of training set 0.993504
Accuracy of validation set 0.966667


In [13]:
from platform import python_version

print(python_version())

3.8.3


In [14]:
import tensorflow as tf
print(tf.__version__)

2.5.0
